In [3]:
import cv2 as cv
import numpy as np
import tkinter as tk
from tkinter import filedialog
from collections import deque

# Initialize Tkinter
root = tk.Tk()
root.withdraw()  # Hide Tkinter Root

# Uploading Image
file_path = filedialog.askopenfilename()
if not file_path:
    print("No Image Selected")
    exit()

# Loading Image
image = cv.imread(file_path)
if image is None:
    print("Can't Load Iamge")
    exit()

original_image = image.copy()  # Prserve Original Image

# Mouse State Dictionary
mouse_state = {'dragged': False, 'start_xy': (-1, -1), 'end_xy': (-1, -1), 'changed': False}

# Image Stack(Max. 10)
image_stack = deque(maxlen=10)
image_stack.append(original_image.copy())

# Create Window
cv.namedWindow('Delete Text')

# Mouse Event Handler Function
def mouse_event_handler(event, x, y, flags, param):
    global original_image

    if event == cv.EVENT_LBUTTONDOWN:
        param['dragged'] = True
        param['start_xy'] = (x, y)
        param['end_xy'] = (x, y)
    elif event == cv.EVENT_LBUTTONUP:
        param['dragged'] = False
        param['end_xy'] = (x, y)
        param['changed'] = True
    elif event == cv.EVENT_MOUSEMOVE and param['dragged']:
        param['end_xy'] = (x, y)

    # Rectangle when Dragging
    if param['dragged']:
        temp_image = original_image.copy()
        x0, y0 = param['start_xy']
        x1, y1 = param['end_xy']
        x_min, x_max = min(x0, x1), max(x0, x1)
        y_min, y_max = min(y0, y1), max(y0, y1)
        cv.rectangle(temp_image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv.imshow('Delete Text', temp_image)

# Initial Image
cv.imshow('Delete Text', original_image)

# Mouse Click Function
cv.setMouseCallback('Delete Text', mouse_event_handler, mouse_state)

while True:
    # Key Event
    key = cv.waitKey(30)
    if key == 27: # ESC
        break
    elif key == ord(' ') and mouse_state['changed']: # Remove Text With Space Bar
        x0, y0 = mouse_state['start_xy']
        x1, y1 = mouse_state['end_xy']

        # Square Of Dragged Region
        x_min, x_max = min(x0, x1), max(x0, x1)
        y_min, y_max = min(y0, y1), max(y0, y1)

        # Set Mask About Region
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        mask[y_min:y_max, x_min:x_max] = 255

        # Mask Dilate
        kernel = np.ones((5, 5), np.uint8)
        dilated_mask = cv.dilate(mask, kernel, iterations=1)

        # Impaint
        inpainted_image = cv.inpaint(original_image, dilated_mask, inpaintRadius=3, flags=cv.INPAINT_TELEA)

        # Update Image
        original_image[y_min:y_max, x_min:x_max] = inpainted_image[y_min:y_max, x_min:x_max]

        # Save To Stack
        image_stack.append(original_image.copy())

        # Initialize 'changed' State
        mouse_state['changed'] = False

        # Show Image
        cv.imshow('Delete Text', original_image)

    elif key == ord('z'): # Redo With 'z'
        if len(image_stack) > 1:
            # Remove Current Image From Stack
            image_stack.pop()
            # Get Previous Image
            original_image = image_stack[-1].copy()
            # Show Image
            cv.imshow('Delete Text', original_image)

    elif key == ord('s'): # Save With 's'
        filename = 'output.png'
        cv.imwrite(filename, original_image)
        print(f"Saved current image as {filename}")

# Destroy All Windows
cv.destroyAllWindows()

Saved current image as output.png
